# Carga de Datos


## .xls / HTML a .csv
Los datos del Mineduc se exportan como archivos .xls, realmente teniendo contenido de HTML. Debido a eso, debemos de parsear el archivo HTML e identificar la tabla correcta que contiene los datos. Luego, podemos exportar los datos a su archivo .csv correspondiente, utilizando el valor de la columna 'Departamento' para nombrarlo. Adicionalmente, los archivos utilizan un encoding diferente al estándar utf-8, por lo cual vamos a especificarlo al momento de leer los archivos HTML.

In [650]:
import pandas as pd
from pathlib import Path
from collections import defaultdict
import re


def parse_html_excel_file(file_path):
    file_path = Path(file_path)

    try:
        tables = pd.read_html(str(file_path), encoding="iso-8859-1")

        if not tables:
            return {"success": False, "error": "No tables found in HTML"}

        required_headers = ["CODIGO", "DISTRITO", "DEPARTAMENTO", "MUNICIPIO"]
        target_table = None
        target_index = None

        for i, df in enumerate(tables):
            df_columns_upper = [str(col).upper().strip() for col in df.columns]
            if all(header in df_columns_upper for header in required_headers):
                target_table = df
                target_index = i
                break
            else:
                if len(df) > 0:
                    first_row_upper = [
                        str(cell).upper().strip() for cell in df.iloc[0]
                    ]
                    if all(header in first_row_upper for header in required_headers):
                        df.columns = df.iloc[0]
                        df = df.drop(df.index[0]).reset_index(drop=True)
                        target_table = df
                        target_index = i
                        break

        if target_table is None:
            return {
                "success": False,
                "error": "No table found with required headers.",
            }

        target_table = target_table.dropna(how="all")

        return {
            "success": True,
            "data": target_table,
            "table_index": target_index,
            "total_tables": len(tables),
        }

    except Exception as e:
        return {"success": False, "error": str(e)}


def sanitize_filename(text):
    filename = text.lower().replace(" ", "_")
    return re.sub(r"[^\w_.]", "", filename)


def process_html_files_directory(input_dir, output_dir):
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    files = list(input_path.glob("*.xls"))

    print(f"Found {len(files)} .xls files to process")

    successful_files = []
    failed_files = []
    all_departamentos = defaultdict(list)

    for file_path in files:
        result = parse_html_excel_file(file_path)

        if result["success"]:
            df = result["data"]

            departamentos = []
            for departamento, group in df.groupby("DEPARTAMENTO"):
                filename = f"datos_{sanitize_filename(departamento)}.csv"

                output_file = output_path / filename
                group.to_csv(output_file, index=False)

                departamentos.append(
                    {"name": departamento, "filename": filename, "rows": len(group)}
                )

                all_departamentos[departamento].append(
                    {
                        "source_file": file_path.name,
                        "csv_file": filename,
                        "rows": len(group),
                    }
                )

            successful_files.append(
                {
                    "file": file_path.name,
                    "departamentos": departamentos,
                    "total_rows": len(df),
                }
            )

            print(f"  ✅ Processed {file_path.name}: Created {len(departamentos)} CSVs.")

        else:
            failed_files.append({"file": file_path.name, "error": result["error"]})
            print(f"  ❌ Failed {file_path.name}: {result['error']}")

    print("\n" + "=" * 60)
    print("PROCESSING SUMMARY")
    print("=" * 60)

    print(f"\nTotal files: {len(files)}")
    print(f"✅ Successful: {len(successful_files)}")
    print(f"❌ Failed: {len(failed_files)}")

    # The detailed successful/failed files lists will only be printed if there are successful/failed files.
    # The summary already gives a count, so the repetition for successful files is removed here.
    # The detailed list for failed files remains as it provides useful error messages.
    if failed_files:
        print(f"\n❌ FAILED FILES:")
        for item in failed_files:
            print(f"  - {item['file']}: {item['error']}")

    duplicates = {
        name: sources
        for name, sources in all_departamentos.items()
        if len(sources) > 1
    }
    if duplicates:
        print(f"\n⚠️ DUPLICATE DEPARTAMENTOS:")
        for dept_name, sources in duplicates.items():
            print(f"  {dept_name}: appears in {len(sources)} files")
    else:
        print("\n✅ No duplicate departamentos found")


test_result = parse_html_excel_file("data/raw/establecimiento.xls")
if test_result["success"]:
    print("\nSingle file test successful!")

    print("\n" + "=" * 60)
    print("PROCESSING ALL FILES")
    print("=" * 60)
    process_html_files_directory("data/raw", "data/csv")
else:
    print(f"❌ Single file test failed: {test_result['error']}")


Single file test successful!

PROCESSING ALL FILES
Found 23 .xls files to process
  ✅ Processed establecimiento (1).xls: Created 1 CSVs.
  ✅ Processed establecimiento (22).xls: Created 1 CSVs.
  ✅ Processed establecimiento (6).xls: Created 1 CSVs.
  ✅ Processed establecimiento (3).xls: Created 1 CSVs.
  ✅ Processed establecimiento (8).xls: Created 1 CSVs.
  ✅ Processed establecimiento (17).xls: Created 1 CSVs.
  ✅ Processed establecimiento (11).xls: Created 1 CSVs.
  ✅ Processed establecimiento (5).xls: Created 1 CSVs.
  ✅ Processed establecimiento (15).xls: Created 1 CSVs.
  ✅ Processed establecimiento (4).xls: Created 1 CSVs.
  ✅ Processed establecimiento.xls: Created 1 CSVs.
  ✅ Processed establecimiento (7).xls: Created 1 CSVs.
  ✅ Processed establecimiento (21).xls: Created 1 CSVs.
  ✅ Processed establecimiento (9).xls: Created 1 CSVs.
  ✅ Processed establecimiento (14).xls: Created 1 CSVs.
  ✅ Processed establecimiento (16).xls: Created 1 CSVs.
  ✅ Processed establecimiento (12)

## .csv a DataFrames
Luego de haber creado los archivos, podemos cargarlos a DataFrames para realizar el análisis necesario.

In [651]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from collections import Counter, defaultdict
import matplotlib.pyplot as plt

# Load all CSV files
csv_dir = Path("data/csv")
csv_files = list(csv_dir.glob("*.csv"))

print(f"Found {len(csv_files)} CSV files")

# Load all datasets
datasets = {}
for csv_file in csv_files:
    dataset_name = csv_file.stem
    datasets[dataset_name] = pd.read_csv(csv_file)

print(f"Loaded {len(datasets)} datasets")

Found 23 CSV files
Loaded 23 datasets


# Descripción

## Filas y Columnas

In [652]:
shape_info = []
for name, df in datasets.items():
    shape_info.append({
        'dataset': name,
        'rows': df.shape[0],
        'columns': df.shape[1]
    })

shape_df = pd.DataFrame(shape_info)
print("📊 Dataset Shapes:")
print(shape_df.sort_values('rows', ascending=False))

print(f"\n📈 Summary:")
print(f"Total rows across all datasets: {shape_df['rows'].sum():,}")
print(f"Average rows per dataset: {shape_df['rows'].mean():.0f}")
print(f"Min/Max rows: {shape_df['rows'].min()} / {shape_df['rows'].max()}")

📊 Dataset Shapes:
                 dataset  rows  columns
14       datos_guatemala  1036       17
4   datos_ciudad_capital   864       17
22      datos_san_marcos   431       17
5        datos_escuintla   393       17
19  datos_quetzaltenango   365       17
21   datos_chimaltenango   300       17
3          datos_jutiapa   296       17
7    datos_suchitepequez   296       17
15   datos_huehuetenango   295       17
10    datos_alta_verapaz   294       17
18          datos_izabal   273       17
9       datos_retalhuleu   272       17
17           datos_peten   270       17
1     datos_sacatepequez   208       17
11          datos_quiche   184       17
6       datos_chiquimula   136       17
13      datos_santa_rosa   133       17
12          datos_jalapa   121       17
16          datos_solola   111       17
2      datos_el_progreso    97       17
20    datos_baja_verapaz    94       17
8           datos_zacapa    70       17
0      datos_totonicapan    51       17

📈 Summary:
Total rows

## Integridad de los Datos

### Consistencia en Nombres de Columnas

In [653]:
all_columns = []
column_consistency = {}

for name, df in datasets.items():
    columns = list(df.columns)
    all_columns.append(columns)
    column_consistency[name] = columns

first_columns = all_columns[0]
all_same = all(columns == first_columns for columns in all_columns)

if all_same:
    print(f"\n✅ Standard columns ({len(first_columns)}):")
    for i, col in enumerate(first_columns, 1):
        print(f"  {i:2d}. {col}")
else:
    print("\n❌ Column differences found:")
    for name, columns in column_consistency.items():
        if columns != first_columns:
            print(f"  {name}: {columns}")


✅ Standard columns (17):
   1. CODIGO
   2. DISTRITO
   3. DEPARTAMENTO
   4. MUNICIPIO
   5. ESTABLECIMIENTO
   6. DIRECCION
   7. TELEFONO
   8. SUPERVISOR
   9. DIRECTOR
  10. NIVEL
  11. SECTOR
  12. AREA
  13. STATUS
  14. MODALIDAD
  15. JORNADA
  16. PLAN
  17. DEPARTAMENTAL


### Encoding Problemático
Como mencionamos anteriormente, el encoding de los archivos originales era distinto de "utf-8". Nos dimos cuenta al realizar el análisis sobre el encoding problemático, sin embargo al cambiarlo dentro de la función anterior logramos correr con éxito este análisis.

In [654]:
import pandas as pd
from collections import defaultdict
import re

problematic_char = "�"
all_problematic_samples = defaultdict(list)
issue_found = False

for dataset_name, df in datasets.items():
    for col in df.columns:
        if df[col].dtype == "object":
            str_series = df[col].astype(str)

            contains_char_mask = str_series.str.contains(problematic_char, na=False)

            if contains_char_mask.any():
                issue_found = True
                current_samples = str_series[contains_char_mask].unique().tolist()
                for sample_val in current_samples:
                    if len(all_problematic_samples[col]) < 5:
                        all_problematic_samples[col].append(sample_val)


if issue_found:
    print(f"❌ Encoding issues found (char: '{problematic_char}'):")
    sorted_cols_with_issues = sorted(all_problematic_samples.keys())

    for col in sorted_cols_with_issues:
        samples = all_problematic_samples[col]
        print(f"  Column '{col}':")
        for val in samples:
            print(f"    • {val}")
else:
    print(f"✅ No encoding issues found (char: '{problematic_char}')")

✅ No encoding issues found (char: '�')


# Análisis de Variables
Las variables que más operaciones de limpieza necesitan son:

In [655]:
summary_stats = []

for col in first_columns:
    col_data = []
    for name, df in datasets.items():
        if col in df.columns:
            series = df[col].astype(str).str.strip()
            col_data.extend(series)

    series_all = pd.Series(col_data)
    n_total = len(series_all)
    n_missing = (series_all == "").sum() + series_all.isna().sum()
    n_unique = series_all.nunique()

    summary_stats.append({
        "column": col,
        "missing (%)": round((n_missing / n_total) * 100, 2),
        "unique_values": n_unique,
        "sample_values": series_all.dropna().unique()[:5].tolist()
    })

df_summary = pd.DataFrame(summary_stats)
df_summary.sort_values("missing (%)", ascending=False, inplace=True)

display(df_summary)

,column,missing (%),unique_values,sample_values
0,CODIGO,0.0,6590,"[08-01-0106-46, 08-01-0107-46, 08-01-0203-46, ..."
1,DISTRITO,0.0,620,"[08-001, 08-024, 08-003, 08-025, 08-007]"
2,DEPARTAMENTO,0.0,23,"[TOTONICAPAN, SACATEPEQUEZ, EL PROGRESO, JUTIA..."
3,MUNICIPIO,0.0,343,"[TOTONICAPAN, SAN CRISTOBAL TOTONICAPAN, SAN F..."
4,ESTABLECIMIENTO,0.0,3779,[ESCUELA NORMAL NACIONAL RURAL DE OCCIDENTE GU...
5,DIRECCION,0.0,4428,"[ZONA 5 PARAJE PARRAMON, 3A. CALLE 12-08, ZONA..."
6,TELEFONO,0.0,4208,"[77661038, 77661920, 77667184, 59238825, 57377..."
7,SUPERVISOR,0.0,598,"[MIGUEL AJPOP VÁSQUEZ, MIGUEL AJPOP VASQUEZ, O..."
8,DIRECTOR,0.0,3860,"[OTTO GILBERTO AMADO TELLO, JOSÉ LUIS IXCAQUIC..."
9,NIVEL,0.0,1,[DIVERSIFICADO]


## Código
Este valor parece ser un identificador único, queremos explorar las siguientes propiedades:
- Unicidad: Este código es único dentro de su respectivo dataset o todos?
- Formato: Es consistente el formato en todos los datasets? Existen errores de digitación?
- Valores Faltantes: Existen valores faltantes?
- Nos ayuda a identificar únicamente a los demás valores?

Debido a esto, queremos realizar los siguientes pasos de limpieza:

- Revisar errores de digitación, cómo lo pueden ser whitespaces o formato incongruente
- Identificar Valores Faltantes
- Identificar unicidad del código
- Identificar si cada código corresponde a un set de valores único de las demás columnas

## Distrito
Este valor parece ser un identificador geográfico, siguiendo un formato XX-YYY. Queremos explorar las siguientes propiedades
- Formato: Es consistente el formato?
- Valores Faltantes: Existen valores faltantes dentro de los datasets?
- Nos ayuda a identificar únicamente algún otro valor?

Debido a esto, queremos realizar los siguientes pasos de limpieza:

- Identificar valores faltantes
- Revisar errores de digitación
- Enforzar un formato consistente
- Identificar si nos puede ayuda a verificar la consistencia de Municipio o algún otro valor

## Departamento

Ya que los DataFrames se encuentran divididos por departamento, esta entrada debería ser completamente consistente. Además, podemos proponer los siguientes pasos para una mayor consistencia:

- Identificar el valor RAW más común, ya que un error de digitación sería menos frecuente
- Tomar ese valor y realizar las siguientes transformaciones
    - Conversión a minúsculas
    - Reemplazo de espacios por _
    - Aplicar a todas las columnas de cada DF individual
- Aplicar OHE luego de mergear los DFs

## Municipio

Este valor representa la división política a nivel municipal. Dado que se usa como categoría geográfica clave, es esencial asegurar consistencia.

Exploración a realizar:

- Formato: ¿Se encuentra todo en mayúsculas? ¿Hay tildes inconsistentes?
- Valores Faltantes: Confirmar si hay celdas vacías o marcadas incorrectamente.
- Relación con otros campos: ¿El municipio concuerda con el departamento correspondiente?

Pasos de limpieza propuestos:

- Convertir todo el texto a mayúsculas (.str.upper()).
- Eliminar espacios extra antes o después (.str.strip()).
- Normalizar tildes y caracteres especiales si necesario.
- Validar los municipios contra una lista oficial por departamento.


## Establecimiento

Nombre propio de la institución educativa. Puede contener muchas variantes tipográficas y estilísticas que dificultan análisis posteriores.

Exploración a realizar:

- Formato: ¿Se mantiene una capitalización consistente?
- Valores Faltantes: ¿Hay registros sin nombre?
- Redundancia o duplicación: ¿Existen nombres duplicados con leves diferencias?

Pasos de limpieza propuestos:

- Capitalizar nombres con .str.title() para uniformidad visual.
- Eliminar espacios repetidos entre palabras.
- Remover puntuación innecesaria.
- Establecer reglas para abreviaciones comunes si se encuentran (ej. "Inst." por "Instituto").

## DIRECCIÓN

Campo libre con alta variabilidad. Las direcciones pueden contener múltiples abreviaturas, puntuación, y errores de digitación.

Exploración a realizar:
- Formato: ¿Existen patrones comunes? ¿Se usan abreviaturas (CALLE, AV, etc.)?
- Valores inconsistentes: ¿Uso indistinto de mayúsculas, puntuación, acentos?

Pasos de limpieza propuestos:
- Normalizar a mayúsculas (.str.upper()).
- Crear reglas de sustitución para abreviaturas frecuentes (ej. AVENIDA → AV.).
- Quitar símbolos innecesarios y estandarizar signos de puntuación.
- Opcional: usar expresiones regulares para separar partes de la dirección (calle, número, zona, etc.).

## TELÉFONO

Campo numérico con alta variabilidad en formato. Puede contener números concatenados, separadores o caracteres no numéricos.

Exploración a realizar:
- Formato: ¿Todos los valores contienen exactamente 8 dígitos? ¿Hay múltiples números por celda?
- Errores: ¿Caracteres no numéricos? ¿Espacios o signos innecesarios?

Pasos de limpieza propuestos:

- Remover todos los caracteres no numéricos con re.sub(r"\D", "", telefono).
- Validar longitud estándar (8 dígitos en Guatemala).
- Si hay múltiples números, dividir y elegir el primero o guardarlos como lista.

## SUPERVISOR

Nombre del supervisor responsable. Puede presentar variaciones por uso de tildes, mayúsculas, y errores ortográficos menores.

Exploración a realizar:

- Formato: ¿Mayúsculas/minúsculas inconsistentes?
- Duplicación: ¿Un mismo nombre aparece escrito de múltiples formas?

Pasos de limpieza propuestos:

- Capitalizar con .str.title() para uniformidad.
- Remover espacios dobles y caracteres extraños.
- Normalizar tildes si es necesario.

## DIRECTOR

Similar al campo de supervisor, representa nombres propios con riesgos similares de inconsistencia.

Exploración a realizar:

- Aplicar los mismos criterios que SUPERVISOR.

Pasos de limpieza propuestos:

- Igual estrategia de capitalización, limpieza de espacios, y normalización de caracteres.

## NIVEL
Este campo tiene un único valor: "DIVERSIFICADO".

Exploración a realizar:

- Verificar que efectivamente todos los valores son iguales.
- Confirmar si es útil conservar esta columna.

Pasos de limpieza propuestos:

- Si es redundante, considerar eliminarla para evitar ruido en análisis futuros.

## SECTOR

Categoría con pocos valores únicos. Se espera valores como "OFICIAL", "PRIVADO", etc.

Exploración a realizar:

- ¿Mayúsculas o minúsculas inconsistentes?
- ¿Errores ortográficos?

Pasos de limpieza propuestos:

- Convertir todo a mayúsculas y eliminar espacios (.str.upper().str.strip()).
- Validar los valores contra un conjunto permitido: {OFICIAL, PRIVADO, MUNICIPAL, COOPERATIVA}.
- Convertir a tipo Categorical.

## ÁREA
Identifica si la institución está en zona rural o urbana.

Exploración a realizar:

- Confirmar que los valores son: URBANA, RURAL, SIN ESPECIFICAR.
- Verificar errores de digitación o combinaciones no válidas.

Pasos de limpieza propuestos:

- Uniformar mayúsculas (.str.upper()).
- Reemplazar variantes de “sin especificar” por un valor estándar (ej. "DESCONOCIDO").
- Convertir a Categorical.

## STATUS
Campo con un único valor: "ABIERTA".

Exploración a realizar:

Verificar si realmente todos los valores son iguales.

Evaluar su utilidad en análisis futuros.

Pasos de limpieza propuestos:

Eliminar si es redundante (sin variabilidad).

## MODALIDAD
Pocos valores únicos: "MONOLINGUE", "BILINGUE".

Exploración a realizar:

Verificar mayúsculas y tildes.

Validar que no existan variantes escritas incorrectamente.

Pasos de limpieza propuestos:

Uniformar mayúsculas y acentos (.str.upper()).

Reemplazar variantes con un mapeo fijo.

Convertir a Categorical

## JORNADA
Categoría horaria. Existen valores como “MATUTINA”, “VESPERTINA”, “DOBLE”, “NOCHE”, etc.

Exploración a realizar:

Verificar consistencia de términos.

Identificar redundancias o términos similares con diferencias menores.

Pasos de limpieza propuestos:

Normalizar formato (.str.upper().str.strip()).

Mapear variantes a un conjunto estándar.

Eliminar signos extra o abreviaciones inconsistentes.

## PLAN
Puede incluir valores con paréntesis, como “DIARIO(REGULAR)”, que dificultan análisis.

Exploración a realizar:

¿Hay signos innecesarios como paréntesis o guiones?

¿Hay términos redundantes?

Pasos de limpieza propuestos:

Eliminar paréntesis y su contenido con str.replace(r'\(.*?\)', '').

Eliminar espacios extra y convertir a mayúsculas.

Validar valores contra una lista limpia predefinida.

Convertir a Categorical.

## DEPARTAMENTAL
Representa una división regional administrativa.

Exploración a realizar:

Verificar mayúsculas, errores tipográficos.

Confirmar que corresponde con el valor del campo DEPARTAMENTO.

Pasos de limpieza propuestos:

Uniformar texto (.str.upper().str.strip()).

Validar contra un listado oficial del MINEDUC.

Opcional: cruzar con DEPARTAMENTO para consistencia.

## Definición de funciones para limpieza de datos

In [656]:
# def clean_establecimiento(df):

#     df_clean = df.copy()

#     df_clean['ESTABLECIMIENTO'] = (df_clean['ESTABLECIMIENTO']
#                                   .str.replace('""', '"', regex=False)
#                                   .str.replace('"', '', regex=False))

#     print(f" ESTABLECIMIENTO limpiado: {(df['ESTABLECIMIENTO'] != df_clean['ESTABLECIMIENTO']).sum()} registros corregidos")

#     return df_clean

# def clean_plan(df):
#     df_clean = df.copy()

#     df_clean['PLAN'] = (df_clean['PLAN']
#                        .str.replace(r'\s*\([^)]*\)', '', regex=True)
#                        .str.strip())

#     print(f" PLAN limpiado: {(df['PLAN'] != df_clean['PLAN']).sum()} registros corregidos")
#     print(f"   Valores únicos después de limpieza: {df_clean['PLAN'].unique()}")

#     return df_clean

# def clean_telefono(df):
#     df_clean = df.copy()

#     def process_phone(phone):
#         if pd.isna(phone) or phone == '':
#             return None

#         phone_str = str(phone)

#         if '-' in phone_str:
#             phone_str = phone_str.split('-')[0]

#         numbers_only = re.sub(r'[^0-9]', '', phone_str)

#         if len(numbers_only) == 8:
#             return numbers_only
#         elif len(numbers_only) == 7:
#             if numbers_only[0] in ['3', '4', '5']:
#                 return '0' + numbers_only
#             else:
#                 return numbers_only
#         else:
#             return None

#     df_clean['TELEFONO'] = df_clean['TELEFONO'].apply(process_phone)

#     valid_phones = df_clean['TELEFONO'].notna().sum()
#     total_phones = len(df_clean)
#     corrected = (df['TELEFONO'].astype(str) != df_clean['TELEFONO'].astype(str)).sum()

#     print(f"TELEFONO limpiado: {corrected} registros corregidos")
#     print(f"   Teléfonos válidos: {valid_phones}/{total_phones} ({valid_phones/total_phones*100:.1f}%)")

#     return df_clean

# def apply_basic_cleaning(df):

#     print(f"🧹 Limpiando dataset con {len(df)} registros...")
#     print("="*50)

#     df_clean = df.copy()

#     df_clean = clean_establecimiento(df_clean)
#     df_clean = clean_plan(df_clean)
#     df_clean = clean_telefono(df_clean)

#     print("="*50)
#     print(f"Limpieza completada")

#     return df_clean



### Generación de encoding categórico

In [657]:
# def create_categorical_encoding(df):
#     df_encoded = df.copy()

#     categorical_fields = ['SECTOR', 'AREA', 'MODALIDAD', 'JORNADA', 'PLAN']

#     print("Creando variables dummy...")

#     for field in categorical_fields:
#         if field in df_encoded.columns:
#             dummies = pd.get_dummies(df_encoded[field], prefix=field.lower())
#             df_encoded = pd.concat([df_encoded, dummies], axis=1)
#             print(f"{field}: {len(dummies.columns)} categorías")

#     df_encoded['es_publico'] = df_encoded['SECTOR'].isin(['OFICIAL', 'MUNICIPAL']).astype(int)
#     df_encoded['es_rural'] = (df_encoded['AREA'] == 'RURAL').astype(int)
#     df_encoded['tiene_telefono'] = df_encoded['TELEFONO'].notna().astype(int)

#     print(f"Variables adicionales: es_publico, es_rural, tiene_telefono")

#     return df_encoded

# def clean_and_prepare_dataset(file_path):
#     print(f"Procesando: {Path(file_path).name}")

#     df = pd.read_csv(file_path)
#     print(f"Registros originales: {len(df)}")

#     df_clean = apply_basic_cleaning(df)

#     df_encoded = create_categorical_encoding(df_clean)

#     print(f"Registros finales: {len(df_encoded)}")
#     print(f"Columnas finales: {len(df_encoded.columns)}")

#     return df_clean, df_encoded

In [658]:
# def process_all_datasets(input_dir="data/csv", output_dir="data/cleaned"):
#     input_path = Path(input_dir)
#     output_path = Path(output_dir)
#     output_path.mkdir(parents=True, exist_ok=True)

#     csv_files = list(input_path.glob("*.csv"))
#     print(f"🚀 Procesando {len(csv_files)} archivos CSV...")
#     print("="*70)

#     all_clean_data = []
#     all_encoded_data = []

#     for csv_file in csv_files:
#         try:
#             df_clean, df_encoded = clean_and_prepare_dataset(csv_file)

#             clean_output = output_path / f"{csv_file.stem}_clean.csv"
#             df_clean.to_csv(clean_output, index=False)

#             all_clean_data.append(df_clean)
#             all_encoded_data.append(df_encoded)

#             print(f"Guardado: {clean_output.name}")

#         except Exception as e:
#             print(f"Error procesando {csv_file.name}: {str(e)}")

#     if all_clean_data:
#         print("\n" + "="*70)
#         print("📊 Creando datasets combinados...")

#         combined_clean = pd.concat(all_clean_data, ignore_index=True)
#         combined_clean_path = output_path / "mineduc_combined_clean.csv"
#         combined_clean.to_csv(combined_clean_path, index=False)

#         combined_encoded = pd.concat(all_encoded_data, ignore_index=True)
#         combined_encoded_path = output_path / "mineduc_combined_encoded.csv"
#         combined_encoded.to_csv(combined_encoded_path, index=False)

#         print(f"✅ Dataset limpio combinado: {len(combined_clean)} registros")
#         print(f"   📁 {combined_clean_path}")
#         print(f"✅ Dataset codificado combinado: {len(combined_encoded)} registros, {len(combined_encoded.columns)} columnas")
#         print(f"   📁 {combined_encoded_path}")

#         return combined_clean, combined_encoded

#     return None, None

# def generate_summary_report(df):
#     print("\n" + "="*60)
#     print("REPORTE FINAL DE DATOS LIMPIOS")
#     print("="*60)

#     print(f"Registros totales: {len(df):,}")
#     print(f"Establecimientos únicos: {df['ESTABLECIMIENTO'].nunique():,}")
#     print(f"Municipios únicos: {df['MUNICIPIO'].nunique()}")
#     print(f"Departamentos únicos: {df['DEPARTAMENTO'].nunique()}")

#     print(f"\nTeléfonos válidos: {df['TELEFONO'].notna().sum():,} ({df['TELEFONO'].notna().mean()*100:.1f}%)")

#     print(f"\nDistribución por SECTOR:")
#     sector_dist = df['SECTOR'].value_counts()
#     for sector, count in sector_dist.items():
#         print(f"   {sector}: {count:,} ({count/len(df)*100:.1f}%)")

#     print(f"\nDistribución por ÁREA:")
#     area_dist = df['AREA'].value_counts()
#     for area, count in area_dist.items():
#         print(f"   {area}: {count:,} ({count/len(df)*100:.1f}%)")

#     print(f"\nDistribución por MODALIDAD:")
#     modalidad_dist = df['MODALIDAD'].value_counts()
#     for modalidad, count in modalidad_dist.items():
#         print(f"   {modalidad}: {count:,} ({count/len(df)*100:.1f}%)")

In [659]:
# df_clean, df_encoded = process_all_datasets()

# if df_clean is not None:
#     generate_summary_report(df_clean)

#     print(f"\n¡Proceso completado exitosamente!")
#     print(f"Archivos disponibles en: data/cleaned/")
#     print(f"- Archivos individuales limpios")
#     print(f"- mineduc_combined_clean.csv (datos limpios)")
#     print(f"- mineduc_combined_encoded.csv (con variables dummy)")
# else:
#     print("No se pudieron procesar los datasets")

In [660]:
# print(df_encoded)











In [661]:


# cleaned_datasets = {}
# for name, df in datasets.items():
#     df_clean = df.copy()

#     df_clean['ESTABLECIMIENTO'] = df_clean['ESTABLECIMIENTO'].str.replace('"', '', regex=False)

#     df_clean['PLAN'] = df_clean['PLAN'].str.replace(r'\s*\([^)]*\)', '', regex=True).str.strip()

#     def clean_phone(phone):
#         if pd.isna(phone):
#             return None
#         phone_str = str(phone)
#         if '-' in phone_str:
#             phone_str = phone_str.split('-')[0]
#         numbers = re.sub(r'[^0-9]', '', phone_str)
#         if len(numbers) == 8:
#             return numbers
#         elif len(numbers) == 7 and numbers[0] in ['3', '4', '5']:
#             return '0' + numbers
#         elif len(numbers) == 7:
#             return numbers
#         else:
#             return None

#     df_clean['TELEFONO'] = df_clean['TELEFONO'].apply(clean_phone)

#     categorical_fields = ['SECTOR', 'AREA', 'MODALIDAD', 'JORNADA', 'PLAN']

#     for field in categorical_fields:
#         if field in df_clean.columns:
#             dummies = pd.get_dummies(df_clean[field], prefix=field.lower())
#             df_clean = pd.concat([df_clean, dummies], axis=1)

#     df_clean['es_publico'] = df_clean['SECTOR'].isin(['OFICIAL', 'MUNICIPAL']).astype(int)
#     df_clean['es_rural'] = (df_clean['AREA'] == 'RURAL').astype(int)
#     df_clean['es_urbano'] = (df_clean['AREA'] == 'URBANA').astype(int)
#     df_clean['tiene_telefono'] = df_clean['TELEFONO'].notna().astype(int)

#     def categorize_establishment(name):
#         name_upper = str(name).upper()
#         if any(word in name_upper for word in ['INSTITUTO', 'COLEGIO']):
#             return 'INSTITUTO_COLEGIO'
#         elif any(word in name_upper for word in ['ESCUELA', 'LICEO']):
#             return 'ESCUELA_LICEO'
#         elif any(word in name_upper for word in ['CENTRO', 'NUCLEO']):
#             return 'CENTRO'
#         else:
#             return 'OTRO'

#     df_clean['tipo_establecimiento'] = df_clean['ESTABLECIMIENTO'].apply(categorize_establishment)
#     df_clean['longitud_nombre'] = df_clean['ESTABLECIMIENTO'].str.len()

#     cleaned_datasets[name] = df_clean

# sample_df = list(cleaned_datasets.values())[0]
# original_cols = 17
# new_cols = len(sample_df.columns)
# print(f"{len(cleaned_datasets)} datasets procesados")
# print(f"Columnas originales: {original_cols}")
# print(f"Columnas finales: {new_cols}")
# print(f"Nuevas columnas creadas: {new_cols - original_cols}")

# all_columns = list(cleaned_datasets[list(cleaned_datasets.keys())[0]].columns)
# summary_stats = []

# for col in all_columns:
#     col_data = []
#     for name, df in cleaned_datasets.items():
#         if col in df.columns:
#             series = df[col].astype(str).str.strip()
#             col_data.extend(series)

#     series_all = pd.Series(col_data)
#     n_total = len(series_all)
#     n_missing = (series_all == "").sum() + series_all.isna().sum() + (series_all == "None").sum()
#     n_unique = series_all.nunique()

#     valid_values = series_all[(series_all != "") & (series_all.notna()) & (series_all != "None")]
#     sample_values = valid_values.unique()[:5].tolist() if len(valid_values) > 0 else []

#     summary_stats.append({
#         "column": col,
#         "missing (%)": round((n_missing / n_total) * 100, 2),
#         "unique_values": n_unique,
#         "sample_values": sample_values
#     })

# df_summary_clean = pd.DataFrame(summary_stats)
# df_summary_clean.sort_values("missing (%)", ascending=False, inplace=True)

# print("RESUMEN COMPLETO DEL DATASET LIMPIO (CON COLUMNAS EXTRAS):")
# display(df_summary_clean)

# print(f"\nIMPACTO DE LA LIMPIEZA:")
# total_records = sum(len(df) for df in cleaned_datasets.values())
# print(f"Total de registros: {total_records:,}")

# combined_clean = pd.concat(cleaned_datasets.values(), ignore_index=True)
# print(f"Teléfonos válidos: {combined_clean['TELEFONO'].notna().sum():,} ({combined_clean['TELEFONO'].notna().mean()*100:.1f}%)")
# print(f"Establecimientos sin comillas: {(~combined_clean['ESTABLECIMIENTO'].str.contains('"', na=False)).sum():,}")
# print(f"Valores únicos en PLAN: {combined_clean['PLAN'].nunique()}")

# Operaciones de Limpieza

## Código

Como primer paso, buscamos identificar errores de digitación. Para esto, vamos a hacer lo siguiente:

- Eliminar los whitespaces dentro de todas las columnas
- Definir un regex que matchee el formato 16-02-0020-46 WW-XX-YYYY-ZZ, para asegurarnos que los datos hayan sido ingresados correctamente.

In [662]:
import re

pattern = r'^\d{2}-\d{2}-\d{4}-\d{2}$'

total_rows = 0
total_valid = 0
total_invalid = 0

for name, df in datasets.items():
    if 'CODIGO' not in df.columns:
        print(f"[{name}] No 'CODIGO' column found.")
        continue

    # Clean whitespace and replace original CODIGO with cleaned version
    df['CODIGO_clean'] = df['CODIGO'].astype(str).str.replace(r'\s+', '', regex=True)
    df['CODIGO'] = df['CODIGO_clean']

    # Check format on cleaned CODIGO
    df['is_valid_format'] = df['CODIGO'].apply(lambda x: bool(re.match(pattern, x)))

    total = len(df)
    valid = df['is_valid_format'].sum()
    invalid = total - valid

    total_rows += total
    total_valid += valid
    total_invalid += invalid

    # Drop helper columns, keep cleaned CODIGO only
    df.drop(columns=['CODIGO_clean', 'is_valid_format'], inplace=True)

print(f"\n=== TOTAL across all datasets ===")
print(f"Total rows: {total_rows} | Valid CODIGO: {total_valid} | Invalid CODIGO: {total_invalid}")



=== TOTAL across all datasets ===
Total rows: 6590 | Valid CODIGO: 6590 | Invalid CODIGO: 0


Luego de haber verificado que cada código sigue el formato correcto, podemos verificar la unicidad del código a lo largo de los diferentes datasets.

In [663]:
import pandas as pd

# Storage for all CODIGO values
all_codigos = []

# Track problems
duplicates_within = {}
missing_codigo = {}

# Analyze each dataset
for name, df in datasets.items():
    if 'CODIGO' not in df.columns:
        print(f"[WARNING] Dataset {name} is missing 'CODIGO' column.")
        continue

    # Clean and standardize CODIGO
    df['CODIGO'] = df['CODIGO'].astype(str).str.strip()

    # Store all codes
    all_codigos.extend(df['CODIGO'].tolist())

    # Check for duplicates within dataset
    duplicated = df['CODIGO'][df['CODIGO'].duplicated()]
    if not duplicated.empty:
        duplicates_within[name] = duplicated.value_counts()

    # Check for missing
    missing = df['CODIGO'].isna().sum() + (df['CODIGO'] == '').sum()
    if missing > 0:
        missing_codigo[name] = missing

# Create a full Series of all codes
codigo_series = pd.Series(all_codigos)

# Check for duplicates across datasets
global_dupes = codigo_series[codigo_series.duplicated(keep=False)]

# Ensure non_unique_codigos_across is always a Series
non_unique_codigos_across = (
    pd.Series(global_dupes.value_counts()) if not global_dupes.empty
    else pd.Series([], dtype=int)
)

# Summary report
print("\n=== Summary ===")
print(f"Total codes across all datasets: {len(codigo_series)}")
print(f"Unique codes: {codigo_series.nunique()}")
print(f"Duplicated across datasets: {len(non_unique_codigos_across)}")

if duplicates_within:
    print("\n❗ Duplicates *within* datasets:")
    for name, series in duplicates_within.items():
        print(f"  - {name}: {series.sum()} duplicates")

if missing_codigo:
    print("\n❗ Missing CODIGO values:")
    for name, count in missing_codigo.items():
        print(f"  - {name}: {count} missing")

if not non_unique_codigos_across.empty:
    print("\n❗ Duplicated CODIGO *across* datasets (top 10):")
    print(non_unique_codigos_across.head(10))



=== Summary ===
Total codes across all datasets: 6590
Unique codes: 6590
Duplicated across datasets: 0


Por último, podemos verificar que cada código corresponda a una y solo una combinación de todas las demás variables.

In [664]:
from collections import defaultdict
import pandas as pd

# Step 1: Combine all datasets into a single DataFrame
all_data = []
for name, df in datasets.items():
    df = df.copy()
    df["source"] = name  # to track origin
    all_data.append(df)
all_df = pd.concat(all_data, ignore_index=True)

# Step 2: Drop CODIGO and group by the remaining columns
grouped = all_df.groupby(all_df.columns.difference(["CODIGO", "source"]).tolist())

# Step 3: Collect entries where multiple CODIGOs share the same other data
duplicates_except_codigo = defaultdict(list)
for _, group in grouped:
    if group["CODIGO"].nunique() > 1:
        duplicates_except_codigo[len(duplicates_except_codigo)] = group

# Step 4: Show sample results
print(f"Found {len(duplicates_except_codigo)} groups with duplicated rows apart from CODIGO.\n")

# Display first few examples
for i, df in list(duplicates_except_codigo.items())[:5]:
    print(f"Group {i+1}:")
    print(df[["CODIGO"] + [col for col in df.columns if col != "CODIGO"]].to_string(index=False))
    print("-" * 60)


Found 134 groups with duplicated rows apart from CODIGO.

Group 1:
       CODIGO DISTRITO DEPARTAMENTO MUNICIPIO                       ESTABLECIMIENTO                         DIRECCION TELEFONO                           SUPERVISOR                           DIRECTOR         NIVEL  SECTOR  AREA  STATUS  MODALIDAD    JORNADA            PLAN DEPARTAMENTAL          source
05-07-0063-46   05-012    ESCUINTLA LA GOMERA COLEGIO MIXTO "LUIS CARDOZA Y ARAGÓN" 4A AVENIDA 5-112 BARRIO CHIPILAPA 78423518 BRENDA YOHANNA GUDIEL LOPEZ DE MUÑOZ KIMBERLY YAMILETH ESPINOZA ESCOBAR DIVERSIFICADO PRIVADO RURAL ABIERTA MONOLINGUE VESPERTINA DIARIO(REGULAR)     ESCUINTLA datos_escuintla
05-07-0066-46   05-012    ESCUINTLA LA GOMERA COLEGIO MIXTO "LUIS CARDOZA Y ARAGÓN" 4A AVENIDA 5-112 BARRIO CHIPILAPA 78423518 BRENDA YOHANNA GUDIEL LOPEZ DE MUÑOZ KIMBERLY YAMILETH ESPINOZA ESCOBAR DIVERSIFICADO PRIVADO RURAL ABIERTA MONOLINGUE VESPERTINA DIARIO(REGULAR)     ESCUINTLA datos_escuintla
------------------------

Esto nos indica que existen entradas duplicadas con códigos diferentes. De momento, este output no nos es realmente útil. Al no tener estandarizadas las demás columnas, puede que todavía haya establecimientos escritos de manera diferente. Sin embargo, esto nos ayuda a determinar que la variable "código" no nos es realmente útil para identificar errores de digitación en otros campos.

## Distrito

## Departamento

El tener datasets separados por departamento nos ayuda un poco con la limpieza, podemos simplemente eliminar trailing / leading whitespaces, convertir a letras mayúsculas e identificar que los valores sean únicos dentro de cada dataset.

In [665]:
departamentos = set()

for name, df in datasets.items():
    if 'DEPARTAMENTO' not in df.columns:
        print(f"[{name}] No 'DEPARTAMENTO' column found.")
        continue

    # Clean in-place: strip whitespace and uppercase
    df['DEPARTAMENTO'] = df['DEPARTAMENTO'].astype(str).str.strip().str.upper()

    # Collect unique cleaned departamentos for checking
    unique_depts = df['DEPARTAMENTO'].unique()
    departamentos.update(unique_depts)

print("Unique DEPARTAMENTO values across all datasets:")
for dept in sorted(departamentos):
    print(f"- {dept}")

Unique DEPARTAMENTO values across all datasets:
- ALTA VERAPAZ
- BAJA VERAPAZ
- CHIMALTENANGO
- CHIQUIMULA
- CIUDAD CAPITAL
- EL PROGRESO
- ESCUINTLA
- GUATEMALA
- HUEHUETENANGO
- IZABAL
- JALAPA
- JUTIAPA
- PETEN
- QUETZALTENANGO
- QUICHE
- RETALHULEU
- SACATEPEQUEZ
- SAN MARCOS
- SANTA ROSA
- SOLOLA
- SUCHITEPEQUEZ
- TOTONICAPAN
- ZACAPA


Verificando los resultados, la limpieza fue exitosa y se comparte un formato consistente e único dentro de cada dataset.

## Municipio

In [666]:
import unicodedata
from difflib import get_close_matches

def normalize_string(s):
    s = str(s).strip().upper()
    s = unicodedata.normalize('NFD', s)
    s = ''.join(c for c in s if unicodedata.category(c) != 'Mn')  # Remove accents
    return s

for name, df in datasets.items():
    if 'MUNICIPIO' not in df.columns or 'DEPARTAMENTO' not in df.columns:
        print(f"[{name}] Missing 'MUNICIPIO' or 'DEPARTAMENTO' column.")
        continue

    # Normalize MUNICIPIO column only
    df['MUNICIPIO'] = df['MUNICIPIO'].apply(normalize_string)

    print(f"\n[{name}] Similar MUNICIPIO names within each DEPARTAMENTO:")

    for dept in df['DEPARTAMENTO'].dropna().unique():
        df_dept = df[df['DEPARTAMENTO'] == dept]
        municipios = df_dept['MUNICIPIO'].dropna().unique()

        checked = set()
        for mun in municipios:
            if mun in checked:
                continue
            matches = get_close_matches(mun, municipios, n=5, cutoff=0.85)
            matches = [m for m in matches if m != mun and m not in checked]
            if matches:
                print(f"- {dept}: {mun} ~ {', '.join(matches)}")
                checked.update(matches)
            checked.add(mun)



[datos_totonicapan] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_sacatepequez] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_el_progreso] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_jutiapa] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_ciudad_capital] Similar MUNICIPIO names within each DEPARTAMENTO:
- CIUDAD CAPITAL: ZONA 1 ~ ZONA 21, ZONA 19, ZONA 18, ZONA 17
- CIUDAD CAPITAL: ZONA 2 ~ ZONA 24, ZONA 12
- CIUDAD CAPITAL: ZONA 3 ~ ZONA 13
- CIUDAD CAPITAL: ZONA 4 ~ ZONA 14
- CIUDAD CAPITAL: ZONA 5 ~ ZONA 15
- CIUDAD CAPITAL: ZONA 6 ~ ZONA 16

[datos_escuintla] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_chiquimula] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_suchitepequez] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_zacapa] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_retalhuleu] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_alta_verapaz] Similar MUNICIPIO names withi

Al ver los resultados, podemos ver que los únicos posibles errores de digitación son las similitudes entre "Zona 1" y "Zona 19" por ejemplo.

## Establecimiento

Primero, podemos empezar "limpiando" el texto al remover caracteres especiales que puede no se hayan incluido en todas las entradas para un mismo establecimiento. Además, vamos a eliminar espacios adicionales por medio trim y trabajar únicamente con mayúsculas.

In [667]:
import unicodedata
import string
from difflib import get_close_matches
import pandas as pd

def clean_text(text):
    """Cleans text in place by:
    1. Removing accents/diacritics
    2. Standardizing punctuation and spaces
    3. Converting to uppercase"""
    if pd.isna(text):
        return text
    text = unicodedata.normalize('NFKD', str(text)).encode('ASCII', 'ignore').decode()
    text = text.translate(str.maketrans('', '', string.punctuation + '"“”‘’'))
    text = ' '.join(text.split())
    return text.upper()

# Clean and analyze in one pass
for name, df in datasets.items():
    if 'MUNICIPIO' not in df.columns or 'ESTABLECIMIENTO' not in df.columns:
        print(f"[{name}] Missing required columns.")
        continue

    # DIRECTLY MODIFY ORIGINAL COLUMNS
    df['MUNICIPIO'] = df['MUNICIPIO'].apply(clean_text)
    df['ESTABLECIMIENTO'] = df['ESTABLECIMIENTO'].apply(clean_text)

    print(f"\n[{name}] Similar ESTABLECIMIENTO names within each MUNICIPIO:")

    # Use the cleaned original columns
    municipios = df['MUNICIPIO'].dropna().unique()
    for mun in municipios:
        df_mun = df[df['MUNICIPIO'] == mun]
        establecimientos = df_mun['ESTABLECIMIENTO'].dropna().unique()

        checked = set()
        matches_found = []

        for est in establecimientos:
            if est in checked:
                continue
            matches = get_close_matches(est, establecimientos, n=5, cutoff=0.95)
            matches = [m for m in matches if m != est and m not in checked]
            if matches:
                matches_found.append((est, matches))
                checked.update(matches)
            checked.add(est)

        if matches_found:
            print(f"\nMunicipio: {mun}")
            for est, matched in matches_found:
                print(f"  Base: {est}")
                print(f"  Similar: {', '.join(matched)}")


[datos_totonicapan] Similar ESTABLECIMIENTO names within each MUNICIPIO:

[datos_sacatepequez] Similar ESTABLECIMIENTO names within each MUNICIPIO:

[datos_el_progreso] Similar ESTABLECIMIENTO names within each MUNICIPIO:

Municipio: SAN AGUSTIN ACASAGUASTLAN
  Base: COLEGIO CENTRO DE FORMACION PROFESIONAL PREUNIVERSITARIA DEL CICLO DIVERSIFICADO
  Similar: COLEGIO CENTRO DE FORMACION PROFESIONAL PREUNIVERISTARIA DEL CICLO DIVERSIFICADO

[datos_jutiapa] Similar ESTABLECIMIENTO names within each MUNICIPIO:

Municipio: JUTIAPA
  Base: ESCUELA EN CIENCIAS DE LA COMUNICACION ECCO II
  Similar: ESCUELA EN CIENCIAS DE LA COMUNICACION ECCO

Municipio: COMAPA
  Base: INSTITUTO PARTICULAR MIXTO DE FORMACION Y DESARROLLO PROFESIONAL INDEFORP
  Similar: INSTITUTO PARTICULAR DE FORMACION Y DESARROLLO PROFESIONAL INDEFORP

Municipio: JALPATAGUA
  Base: COLEGIO PARTICULAR MIXTO SAN JOSE OBRERO II
  Similar: COLEGIO PARTICULAR MIXTO SAN JOSE OBRERO I

Municipio: MOYUTA
  Base: INSTITUTO TECNOLOGICO 

Podemos ver que todavía tenemos múltiples establecimientos con nombres similares, esto lo vamos a dejar así de momento y buscamos utilizar otras variables para identificar un mismo establecimiento escrito de múltiples maneras. En algunos casos, logramos identificar errores pequeños como "NO 3" y "NO3" que indica nombres diferentes. En estos casos, buscamos utilizar otras variables identificadores cómo teléfono y dirección para matchear nombres iguales.

## Dirección

Este campo es bastante complicado, por lo que vamos a tomar un approach iterativo. Es decir, primero vamos a empezar con estandarización básica y luego vamos a revisar direcciones similares dentro de un mismo municipio. Esto nos permitirá ver errores que no habíamos visto anteriormente. Primero, vamos a reemplazar abreviaciones comunes como

- AV - Avenida
- COL - Colonia
- DIAG - DIAGNOAL

In [668]:
import pandas as pd
import re

def clean_address(address):
    if pd.isna(address): return ""
    addr = str(address).upper().strip()

    # Protect kilometer markers
    km_markers = re.findall(r'KM\s*\d+\.?\d*', addr)
    for km in km_markers:
        addr = addr.replace(km, km.replace(' ', '_'))

    # Standard replacements
    replacements = [
        (r',', ''),  # Remove all commas
        (r'\bAV\.?\b', 'AVENIDA'),
        (r'\bC\.?\b', 'CALLE'),
        (r'\bCL\.?\b', 'CALLE'),
        (r'\bDIAG\.?\b', 'DIAGONAL'),
        (r'\bCOL\.?\b', 'COLONIA'),
        (r'(\d+)[A-Za-z]*\.?', r'\1'),
        (r'\.(?!\d)', ''),  # Remove periods not before numbers
        (r'\s+', ' ')
    ]

    for pat, repl in replacements:
        addr = re.sub(pat, repl, addr)

    # Restore KM markers
    for km in km_markers:
        clean_km = km.replace('_', ' ').replace('. ', '.').replace(' ', '')
        addr = addr.replace(km.replace(' ', '_'), clean_km)

    addr = re.sub(r'Z\.?\b', 'ZONA', addr)
    addr = re.sub(r'(\d+)\s*-\s*(\d+)', r'\1-\2', addr)

    return addr

# Apply cleaning DIRECTLY to DIRECCION column
for name, df in datasets.items():
    if 'DIRECCION' in df.columns:
        df['DIRECCION'] = df['DIRECCION'].apply(clean_address)

# Verification: Print original and modified addresses from first 3 datasets
print("=== ADDRESS CLEANING VERIFICATION ===")
for name, df in list(datasets.items())[:3]:
    if 'DIRECCION' in df.columns:
        samples = df[['DIRECCION']].dropna().head(2)
        print(f"\nDataset: {name}")
        for addr in samples['DIRECCION']:
            print(f"  {addr}")

=== ADDRESS CLEANING VERIFICATION ===

Dataset: datos_totonicapan
  ZONA 5 PARAJE PARRAMON
  3 CALLE 12-08 ZONA 2

Dataset: datos_sacatepequez
  5 CALLE ORIENTE NO.17
  3 CALLE ORIENTE NO.11

Dataset: datos_el_progreso
  BARRIO EL CALVARIO
  BARRIO EL PORVENIR


Luego de limpiar las direcciones, podemos utilizar una función para intentar identificar direcciones similares

In [669]:
def find_similar_addresses(datasets, cutoff=0.9):
    repeated = 0
    for name, df in datasets.items():
        if 'MUNICIPIO' not in df.columns or 'DIRECCION' not in df.columns:
            continue

        print(f"\n[{name}] Potential address typos:")

        municipios = df['MUNICIPIO'].dropna().unique()
        for mun in municipios:
            df_mun = df[df['MUNICIPIO'] == mun]
            addresses = df_mun['DIRECCION'].dropna().unique()

            checked = set()
            matches_found = []

            for addr in addresses:
                if addr in checked:
                    continue

                # Standardize connectors for better matching
                standardized = re.sub(r'\b(Y|CON)\b', '', addr)

                # Filter candidates (same street type and zone)
                street_type = re.search(r'(AVENIDA|CALLE|DIAGONAL|COLONIA|KM)', addr)
                zone = re.search(r'ZONA \d+', addr)

                if not street_type or not zone:
                    continue

                candidates = [
                    a for a in addresses
                    if street_type.group() in a
                    and zone.group() in a
                    and a != addr  # Exclude self
                ]

                # Compare standardized versions
                matches = [
                    m for m in candidates
                    if get_close_matches(standardized, [re.sub(r'\b(Y|CON)\b', '', m)], n=1, cutoff=cutoff)
                ]

                if matches:
                    matches_found.append((addr, matches))
                    checked.update(matches)
                    repeated += 1

                checked.add(addr)

            if matches_found:
                print(f"\nMunicipio: {mun}")
                for addr, matched in matches_found:
                    print(f"  {addr}")
                    print(f"  → Possible variants: {', '.join(matched)}\n")
    print(repeated)

# Usage:
find_similar_addresses(datasets)


[datos_totonicapan] Potential address typos:

[datos_sacatepequez] Potential address typos:

Municipio: SAN LUCAS SACATEPEQUEZ
  1 AVENIDA Y 6 CALLE 1-80 ZONA 1
  → Possible variants: 1 AVENIDA 6 CALLE 1-80 ZONA 1


Municipio: CIUDAD VIEJA
  3 CALLE 1-22 ZONA 3
  → Possible variants: 3 CALLE1-22 ZONA 3


[datos_el_progreso] Potential address typos:

Municipio: SANARATE
  2 AVENIDA 1-76 ZONA 4
  → Possible variants: 2 AVENIDA-1-76 ZONA 4, 2 AVENIDA 3-66 ZONA 4


[datos_jutiapa] Potential address typos:

Municipio: JUTIAPA
  7 CALLE 8 AVENIDA 7-49 ZONA 3 BARRIO EL CHAPARRON
  → Possible variants: 7 CALLE Y 8 AVENIDA 7-49 ZONA 3 BARRIO EL CHAPARRON, 7 CALLE Y 8 AVENIDA 7-49 ZONA 3 BARRIO EL CHAPARON

  8 CALLE 4-42 ZONA 1 BARRIO LATINO
  → Possible variants: 8ª CALLE ZONA 1 BARRIO LATINO


Municipio: JALPATAGUA
  7 AVENIDA 4-37 ZONA 1
  → Possible variants: 6 AVENIDA 4-34 ZONA 1

  2 AVE Y 4 CALLE ZONA 2
  → Possible variants: 2 AVENIDA Y 4 CALLE ZONA 2

  3- CALLE 5-97 ZONA 1
  → Possib

Basado en el output, nos podemos dar cuenta que hay algunas modificaciones que podemos realizar todavía:

- Palabras / letras conectivas, por ejemplo "1 AVENIDA Y 6 CALLE 1-80 ZONA 1" y "1 AVENIDA 6 CALLE 1-80 ZONA 1"
- Caracteres especiales para los ordinales, cómo "8ª CALLE ZONA 1 BARRIO LATINO"

Las demás modificaciones presentan bastantes complicaciones al intentar generalizarlas, por lo que terminaremos de limpiar las demás columnas e intentaremos identificar establecimientos con la misma dirección para intervenir "manualmente" y establecer una única dirección.


In [670]:
import re
import unicodedata
import pandas as pd

def refine_address(address):
    if not address or pd.isna(address):
        return address

    # Convert to string and normalize special characters
    addr = str(address)

    # Remove accents and diacritics (like clean_text does)
    addr = unicodedata.normalize('NFKD', addr).encode('ASCII', 'ignore').decode()

    # Handle ordinal indicators (ª, º)
    addr = re.sub(r'(\d+)(ª|º)\b', r'\1', addr)

    # Standardize connectors (Y, CON) - replace with space
    addr = re.sub(r'\b(Y|CON)\b', ' ', addr)

    # Remove other special characters (preserving hyphens between numbers)
    # First protect number-hyphen-number patterns
    protected = re.findall(r'\d+-\d+', addr)
    for i, ph in enumerate(protected):
        addr = addr.replace(ph, f'__PROTECTED_{i}__')

    # Remove punctuation except protected patterns
    addr = re.sub(r'[^\w\s-]', ' ', addr)

    # Restore protected patterns
    for i, ph in enumerate(protected):
        addr = addr.replace(f'__PROTECTED_{i}__', ph)

    # Clean up spaces
    addr = re.sub(r'\s+', ' ', addr).strip().upper()

    return addr

# Apply the additional cleaning to all datasets
for name, df in datasets.items():
    if 'DIRECCION' in df.columns:
        df['DIRECCION'] = df['DIRECCION'].apply(refine_address)

# Verification: Print modified addresses from first 3 datasets
print("\n=== REFINED ADDRESS CLEANING VERIFICATION ===")
for name, df in list(datasets.items())[:3]:
    if 'DIRECCION' in df.columns:
        samples = df[['DIRECCION']].dropna().head(3)
        print(f"\nDataset: {name}")
        for addr in samples['DIRECCION']:
            print(f"  {addr}")


=== REFINED ADDRESS CLEANING VERIFICATION ===

Dataset: datos_totonicapan
  ZONA 5 PARAJE PARRAMON
  3 CALLE 12-08 ZONA 2
  6 AVENIDA 1-47 ZONA 1

Dataset: datos_sacatepequez
  5 CALLE ORIENTE NO 17
  3 CALLE ORIENTE NO 11
  4 AVENIDA NORTE N-26

Dataset: datos_el_progreso
  BARRIO EL CALVARIO
  BARRIO EL PORVENIR
  BARRIO EL PORVENIR


## Telefono

In [671]:
import pandas as pd
import re

def validate_phones(df):
    if 'TELEFONO' not in df.columns:
        return "No phone column"

    valid_count = 0
    invalid_count = 0
    problem_samples = []

    for phone in df['TELEFONO']:
        # Handle NaN/empty first
        if pd.isna(phone):
            invalid_count += 1
            continue

        # Convert to string, strip whitespace
        phone_str = str(phone).strip()

        # Remove .0 if caused by float
        if phone_str.endswith('.0'):
            phone_str = phone_str[:-2]

        # Remove ALL non-digit characters
        digits_only = re.sub(r'[^\d]', '', phone_str)

        # Validate
        if len(digits_only) == 8:
            valid_count += 1
        else:
            invalid_count += 1
            if len(problem_samples) < 3:
                problem_samples.append(phone_str)

    result = f"✅ {valid_count} valid | ❌ {invalid_count} invalid"
    if problem_samples:
        result += f"\n   Sample problems: {problem_samples}"
    return result

# === FULL LOOP TO PRINT RESULTS FOR EACH DF ===
print("=== TRUE PHONE VALIDATION ===")
for name, df in datasets.items():
    if 'TELEFONO' in df.columns:
        print(f"{name.ljust(25)} {validate_phones(df)}")


=== TRUE PHONE VALIDATION ===
datos_totonicapan         ✅ 46 valid | ❌ 5 invalid
   Sample problems: ['56769964-77663283', '56769964-7766328', '77663283-77663283']
datos_sacatepequez        ✅ 207 valid | ❌ 1 invalid
   Sample problems: ['52660000-78323083']
datos_el_progreso         ✅ 93 valid | ❌ 4 invalid
   Sample problems: ['4215928', '3033']
datos_jutiapa             ✅ 287 valid | ❌ 9 invalid
   Sample problems: ['7844007', '7844007']
datos_ciudad_capital      ✅ 860 valid | ❌ 4 invalid
   Sample problems: ['2232068', '2223228', '2232379']
datos_escuintla           ✅ 390 valid | ❌ 3 invalid
   Sample problems: ['78880429-78880419']
datos_chiquimula          ✅ 129 valid | ❌ 7 invalid
datos_suchitepequez       ✅ 288 valid | ❌ 8 invalid
   Sample problems: ['4140069', '4210394058993785', '5899378']
datos_zacapa              ✅ 70 valid | ❌ 0 invalid
datos_retalhuleu          ✅ 264 valid | ❌ 8 invalid
datos_alta_verapaz        ✅ 286 valid | ❌ 8 invalid
   Sample problems: ['79504027-795

Nos podemos dar cuenta de 2 cosas, el principal "error" son los establecimientos con dos números telefónicos. Lo que podemos hacer, es tomar 2 números telefónicos como válidos y si un establecimiento cuenta con 2 números telefónicos mal ingresados (ej. 4210394058993785) separarlos utilizando un -.

In [672]:
import re
import pandas as pd

import re
import pandas as pd

def fix_phones(df):
    if 'TELEFONO' not in df.columns:
        return df

    for i, phone in df['TELEFONO'].items():
        if pd.isna(phone):
            continue

        phone_str = str(phone).strip()

        if phone_str.endswith('.0'):
            phone_str = phone_str[:-2]

        digit_chunks = re.findall(r'\d{5,}', phone_str)

        final_numbers = []

        if len(digit_chunks) == 1 and len(digit_chunks[0]) >= 14:
            candidates = re.findall(r'\d{8}', digit_chunks[0])
            if len(candidates) >= 2:
                final_numbers = candidates[:2]

        else:
            # Solo tomar los números exactos de 8 dígitos
            final_numbers = [n for n in digit_chunks if len(n) == 8][:2]

        if final_numbers:
            df.at[i, 'TELEFONO'] = '-'.join(final_numbers)
        else:
            # En lugar de borrar, dejamos el valor original intacto para que validación lo detecte
            df.at[i, 'TELEFONO'] = phone_str

    return df


# Primero lo arreglás
for name, df in datasets.items():
    if 'TELEFONO' in df.columns:
        fix_phones(df)

/tmp/ipython-input-1746684611.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '79450881' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'TELEFONO'] = '-'.join(final_numbers)
/tmp/ipython-input-1746684611.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '78441241' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'TELEFONO'] = '-'.join(final_numbers)
/tmp/ipython-input-1746684611.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '22512759' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, 'TELEFONO'] = '-'.join(final_numbers)
/tmp/ipython-input-1746684611.py:34: FutureWarning: Setting 

Ahora podemos utilizar una función de validación actualizada, dónde se toman como válidos varios números telefónicos y podemos asegurar que los pendientes son teléfonos faltantes o mal

In [673]:
import pandas as pd
import re

def validate_phones(df):
    if 'TELEFONO' not in df.columns:
        return "No phone column"

    valid_count = 0
    invalid_count = 0
    problem_samples = []

    for phone in df['TELEFONO']:
        if pd.isna(phone) or not str(phone).strip():
            invalid_count += 1
            continue

        phone_str = str(phone).strip()

        # Remove .0 if float artifact
        if phone_str.endswith('.0'):
            phone_str = phone_str[:-2]

        # Split multiple possible phones
        parts = re.split(r'[-/\s]', phone_str)

        part_has_valid = False
        for part in parts:
            digits_only = re.sub(r'\D', '', part)
            if len(digits_only) == 8:
                valid_count += 1
                part_has_valid = True
            elif digits_only:
                invalid_count += 1
                # Add example only if there's at least something wrong but present
                if len(problem_samples) < 3 and phone_str not in problem_samples:
                    problem_samples.append(phone_str)

        if not parts and not part_has_valid:
            invalid_count += 1

    result = f"✅ {valid_count} valid | ❌ {invalid_count} invalid"
    if problem_samples:
        result += f"\n   Sample problems: {problem_samples}"
    return result

print("=== TRUE PHONE VALIDATION ===")
for name, df in datasets.items():
    if 'TELEFONO' in df.columns:
        print(f"{name.ljust(25)} {validate_phones(df)}")


=== TRUE PHONE VALIDATION ===
datos_totonicapan         ✅ 55 valid | ❌ 0 invalid
datos_sacatepequez        ✅ 209 valid | ❌ 0 invalid
datos_el_progreso         ✅ 93 valid | ❌ 4 invalid
   Sample problems: ['4215928', '3033']
datos_jutiapa             ✅ 287 valid | ❌ 9 invalid
   Sample problems: ['7844007']
datos_ciudad_capital      ✅ 860 valid | ❌ 4 invalid
   Sample problems: ['2232068', '2223228', '2232379']
datos_escuintla           ✅ 392 valid | ❌ 2 invalid
datos_chiquimula          ✅ 129 valid | ❌ 7 invalid
datos_suchitepequez       ✅ 294 valid | ❌ 5 invalid
   Sample problems: ['4140069', '5899378']
datos_zacapa              ✅ 70 valid | ❌ 0 invalid
datos_retalhuleu          ✅ 264 valid | ❌ 8 invalid
datos_alta_verapaz        ✅ 288 valid | ❌ 7 invalid
   Sample problems: ['4085613']
datos_quiche              ✅ 183 valid | ❌ 1 invalid
   Sample problems: ['0']
datos_jalapa              ✅ 122 valid | ❌ 0 invalid
datos_santa_rosa          ✅ 132 valid | ❌ 1 invalid
datos_guatemala   

In [674]:
import pandas as pd
import re

def extract_valid_phones(phone_str):
    """Extrae todos los teléfonos válidos (8 dígitos) de un string."""
    if not phone_str or pd.isna(phone_str):
        return []
    phone_str = str(phone_str).strip()
    if phone_str.endswith('.0'):
        phone_str = phone_str[:-2]
    parts = re.split(r'[-/\s]', phone_str)
    valid_phones = []
    for part in parts:
        digits_only = re.sub(r'\D', '', part)
        if len(digits_only) == 8:
            valid_phones.append(digits_only)
    return valid_phones

def build_valid_phone_reference(datasets):
    """Crea un diccionario {ESTABLECIMIENTO: lista de teléfonos válidos únicos} de todos los datasets."""
    phone_ref = {}
    for df in datasets.values():
        if all(col in df.columns for col in ['ESTABLECIMIENTO', 'TELEFONO']):
            for _, row in df.iterrows():
                name = row['ESTABLECIMIENTO']
                if pd.isna(name):
                    continue
                phones = extract_valid_phones(row['TELEFONO'])
                if not phones:
                    continue
                if name not in phone_ref:
                    phone_ref[name] = set()
                phone_ref[name].update(phones)
    # Convertir sets a listas para consistencia
    for k in phone_ref:
        phone_ref[k] = list(phone_ref[k])
    return phone_ref

def fix_invalid_phones_by_reference(datasets):
    print("=== FIXING INVALID PHONES BY ESTABLECIMIENTO REFERENCE ===")
    phone_ref = build_valid_phone_reference(datasets)
    fixed_count = 0

    for name, df in datasets.items():
        if all(col in df.columns for col in ['ESTABLECIMIENTO', 'TELEFONO']):
            for i, row in df.iterrows():
                phone_val = row['TELEFONO']
                est_name = row['ESTABLECIMIENTO']
                if pd.isna(phone_val) or not str(phone_val).strip():
                    # teléfono vacío o nulo, intentar reemplazar
                    if est_name in phone_ref:
                        new_phone = '-'.join(phone_ref[est_name])
                        df.at[i, 'TELEFONO'] = new_phone
                        fixed_count += 1
                else:
                    # teléfono presente, validar
                    valid_phones = extract_valid_phones(phone_val)
                    # Si no tiene ningún teléfono válido, intentar reemplazar
                    if len(valid_phones) == 0 and est_name in phone_ref:
                        new_phone = '-'.join(phone_ref[est_name])
                        df.at[i, 'TELEFONO'] = new_phone
                        fixed_count += 1

    print(f"Total teléfonos corregidos: {fixed_count}")
    return datasets

# Uso:
datasets = fix_invalid_phones_by_reference(datasets)

# Luego validás con tu función habitual
print("=== TRUE PHONE VALIDATION ===")
for name, df in datasets.items():
    if 'TELEFONO' in df.columns:
        print(f"{name.ljust(25)} {validate_phones(df)}")


=== FIXING INVALID PHONES BY ESTABLECIMIENTO REFERENCE ===
Total teléfonos corregidos: 47
=== TRUE PHONE VALIDATION ===
datos_totonicapan         ✅ 55 valid | ❌ 0 invalid
datos_sacatepequez        ✅ 209 valid | ❌ 0 invalid
datos_el_progreso         ✅ 474 valid | ❌ 1 invalid
   Sample problems: ['3033']
datos_jutiapa             ✅ 652 valid | ❌ 5 invalid
datos_ciudad_capital      ✅ 866 valid | ❌ 0 invalid
datos_escuintla           ✅ 392 valid | ❌ 2 invalid
datos_chiquimula          ✅ 1213 valid | ❌ 3 invalid
datos_suchitepequez       ✅ 300 valid | ❌ 1 invalid
datos_zacapa              ✅ 70 valid | ❌ 0 invalid
datos_retalhuleu          ✅ 270 valid | ❌ 2 invalid
datos_alta_verapaz        ✅ 1016 valid | ❌ 1 invalid
datos_quiche              ✅ 544 valid | ❌ 0 invalid
datos_jalapa              ✅ 122 valid | ❌ 0 invalid
datos_santa_rosa          ✅ 133 valid | ❌ 0 invalid
datos_guatemala           ✅ 1402 valid | ❌ 1 invalid
   Sample problems: ['40']
datos_huehuetenango       ✅ 295 valid | ❌ 0

In [675]:
import pandas as pd
import re

def validate_phones(df):
    if 'TELEFONO' not in df.columns:
        return "No phone column"

    valid_count = 0
    invalid_count = 0
    problem_samples = []

    for phone in df['TELEFONO']:
        if pd.isna(phone) or not str(phone).strip():
            invalid_count += 1
            if len(problem_samples) < 3:
                problem_samples.append('')
            continue

        phone_str = str(phone).strip()

        # Remove .0 if float artifact
        if phone_str.endswith('.0'):
            phone_str = phone_str[:-2]

        # Split if there are multiple numbers
        parts = re.split(r'[-/\s]', phone_str)

        part_has_valid = False
        for part in parts:
            digits_only = re.sub(r'\D', '', part)
            if len(digits_only) == 8:
                valid_count += 1
                part_has_valid = True
            elif digits_only:
                invalid_count += 1
                if len(problem_samples) < 3 and phone_str not in problem_samples:
                    problem_samples.append(phone_str)

        if not part_has_valid and not parts:
            # fallback case: string with no valid digits
            invalid_count += 1
            if len(problem_samples) < 3 and phone_str not in problem_samples:
                problem_samples.append(phone_str)

    result = f"✅ {valid_count} valid | ❌ {invalid_count} invalid"
    if problem_samples:
        result += f"\n   Sample problems: {problem_samples}"
    return result

print("=== TRUE PHONE VALIDATION ===")
for name, df in datasets.items():
    if 'TELEFONO' in df.columns:
        print(f"{name.ljust(25)} {validate_phones(df)}")


=== TRUE PHONE VALIDATION ===
datos_totonicapan         ✅ 55 valid | ❌ 0 invalid
datos_sacatepequez        ✅ 209 valid | ❌ 0 invalid
datos_el_progreso         ✅ 474 valid | ❌ 1 invalid
   Sample problems: ['3033']
datos_jutiapa             ✅ 652 valid | ❌ 5 invalid
   Sample problems: ['', '', '']
datos_ciudad_capital      ✅ 866 valid | ❌ 0 invalid
datos_escuintla           ✅ 392 valid | ❌ 2 invalid
   Sample problems: ['', '']
datos_chiquimula          ✅ 1213 valid | ❌ 3 invalid
   Sample problems: ['', '', '']
datos_suchitepequez       ✅ 300 valid | ❌ 1 invalid
   Sample problems: ['']
datos_zacapa              ✅ 70 valid | ❌ 0 invalid
datos_retalhuleu          ✅ 270 valid | ❌ 2 invalid
   Sample problems: ['', '']
datos_alta_verapaz        ✅ 1016 valid | ❌ 1 invalid
   Sample problems: ['']
datos_quiche              ✅ 544 valid | ❌ 0 invalid
datos_jalapa              ✅ 122 valid | ❌ 0 invalid
datos_santa_rosa          ✅ 133 valid | ❌ 0 invalid
datos_guatemala           ✅ 1402 valid 

## Supervisor

Para esta variable, realmente lo único que podemos hacer es limpiarla utilizando estandarización como letras mayúsculas. Si intentamos utilizar algunas de las estrategias anteriores, como fuzzy matching al nombre, realmente no tenemos manera de identificar que "Juan Prez" es un error de digitación para "Juan Perez".

In [676]:
import unicodedata
import string

def clean_supervisor_name(name):
    """Cleans a supervisor name by:
    1. Converting to uppercase
    2. Removing accents/diacritics
    3. Stripping whitespace
    4. Removing punctuation"""
    if pd.isna(name):
        return None

    # Convert to string, uppercase, and strip whitespace
    name = str(name).upper().strip()

    # Remove accents/diacritics
    name = unicodedata.normalize('NFKD', name)\
           .encode('ASCII', 'ignore')\
           .decode('ASCII')

    # Remove punctuation (keeps only letters and spaces)
    name = name.translate(str.maketrans('', '', string.punctuation))

    # Collapse multiple spaces
    name = ' '.join(name.split())

    return name

# Apply cleaning to SUPERVISOR column in all datasets
for df in datasets.values():
    if 'SUPERVISOR' in df.columns:
        df['SUPERVISOR'] = df['SUPERVISOR'].apply(clean_supervisor_name)

# Verification print
print("SUPERVISOR cleaning completed for all datasets.")
print("Sample cleaned names:")
for name, df in list(datasets.items())[:2]:  # Show first 2 datasets
    if 'SUPERVISOR' in df.columns:
        print(f"\n{name}:")
        print(df['SUPERVISOR'].dropna().unique()[:5])  # Show first 5 unique names

SUPERVISOR cleaning completed for all datasets.
Sample cleaned names:

datos_totonicapan:
['MIGUEL AJPOP VASQUEZ' 'OSCAR MOISES LACAN ALVAREZ'
 'VICTOR JESUS TZIC LACAN' 'YESENIA CAROLINA TACAM MORALES'
 'HEBERTO WOTZBELI PALACIOS JUC']

datos_sacatepequez:
['JUAN DEMETRIO SICAJOL PEREZ' 'ZOILA ESTHELA JONFE OROZCO'
 'AURA MARINA LORENZO SAGCHE' 'OSCAR OSWALDO SON CAMEZ'
 'JUVENTINO PEREZ SICAN']


## Director

De manera similar a la anterior,  realmente lo único que podemos hacer es limpiarla utilizando estandarización como letras mayúsculas. Si intentamos utilizar algunas de las estrategias anteriores, como fuzzy matching al nombre, realmente no tenemos manera de identificar que "Juan Prez" es un error de digitación para "Juan Perez".

In [677]:
import unicodedata
import string

def clean_director_name(name):
    """Cleans a director name by:
    1. Converting to uppercase
    2. Removing accents/diacritics
    3. Stripping whitespace
    4. Removing punctuation"""
    if pd.isna(name):
        return None

    # Convert to string, uppercase, and strip whitespace
    name = str(name).upper().strip()

    # Remove accents/diacritics
    name = unicodedata.normalize('NFKD', name)\
           .encode('ASCII', 'ignore')\
           .decode('ASCII')

    # Remove punctuation (keeps only letters and spaces)
    name = name.translate(str.maketrans('', '', string.punctuation))

    # Collapse multiple spaces
    name = ' '.join(name.split())

    return name

# Apply cleaning to DIRECTOR column in all datasets
for df in datasets.values():
    if 'DIRECTOR' in df.columns:
        df['DIRECTOR'] = df['DIRECTOR'].apply(clean_director_name)

# Verification print
print("DIRECTOR cleaning completed for all datasets.")
print("Sample cleaned names:")
for name, df in list(datasets.items())[:2]:  # Show first 2 datasets
    if 'DIRECTOR' in df.columns:
        print(f"\n{name}:")
        print(df['DIRECTOR'].dropna().unique()[:5])  # Show first 5 unique names


DIRECTOR cleaning completed for all datasets.
Sample cleaned names:

datos_totonicapan:
['OTTO GILBERTO AMADO TELLO' 'JOSE LUIS IXCAQUIC AGUILAR'
 'JUAN GILBERTO TZIC TZIC' 'JESUS CRISTINA LOPEZ TZOC'
 'HIGINIO DAVID JUAREZ RODRIGUEZ']

datos_sacatepequez:
['JULIO ABDIEL HERNANDEZ GARCIA' 'JOSUE NATHAN SOLORZANO BERDUO'
 'MARCOS MANUEL HERNANDEZ PEREZ' 'SONIA JUDITH GALVEZ GARCIA'
 'MANUEL ANTONIO ESTRADA ORTIZ']


## Consistencia Establecimiento, Dirección y Teléfono
Hasta este punto, hemos estado utilizando diferentes estrategias para limpiar cada variable por separado. Sin embargo, todavía debemos intentar identificar posibles errores de digitación utilizando diferentes variables en conjunto.

Primero, vamos a identificar pequeños errores de digitación para los nombres de establecimiento. Estaremos utilizando fuzzy matching con un treshold de 0.95. Este fue decidido luego de algunas pruebas y errores en un inciso anterior, dónde algunos institutos diferentes aparecían dentro de la lista al tener una similitud en ej. "Primaria" vs "Preprimaria". Vamos a utilizar matching del 0.95, luego para terminar de identificar el establecimiento estaremos utilizando número telefónico y director. D

In [678]:
from difflib import get_close_matches
from collections import defaultdict
import pandas as pd

def corregir_nombres_similares(datasets, threshold=0.95):
    """
    Corrige nombres de establecimientos similares (pero no idénticos) para cada grupo con el mismo teléfono y director.
    """

    # Paso 1: Agrupar por teléfono + director
    grupos = defaultdict(list)
    for nombre_df, df in datasets.items():
        if all(col in df.columns for col in ['ESTABLECIMIENTO', 'TELEFONO', 'DIRECTOR']):
            for idx, row in df.iterrows():
                if pd.notna(row['TELEFONO']) and pd.notna(row['DIRECTOR']):
                    clave = (str(row['TELEFONO']), str(row['DIRECTOR']).upper())
                    nombre = str(row['ESTABLECIMIENTO']).upper().strip()
                    grupos[clave].append((nombre_df, idx, nombre))

    # Paso 2: Crear un mapeo de corrección
    reemplazos = {}

    for clave, lista in grupos.items():
        nombres_unicos = list(set([nombre for _, _, nombre in lista]))
        clusters = []

        for nombre in nombres_unicos:
            # Intentar meter el nombre en un cluster existente
            found_cluster = False
            for cluster in clusters:
                if get_close_matches(nombre, cluster, n=1, cutoff=threshold):
                    cluster.append(nombre)
                    found_cluster = True
                    break
            if not found_cluster:
                clusters.append([nombre])

        # Para cada cluster, seleccionar el nombre canónico y mapear los demás
        for cluster in clusters:
            if len(cluster) < 2:
                continue  # No hay errores si sólo hay uno
            nombre_referencia = max(cluster, key=len)  # El más largo como referencia
            for variante in cluster:
                if variante != nombre_referencia:
                    reemplazos[variante] = nombre_referencia

    # Paso 3: Reemplazar en los datasets
    for df in datasets.values():
        if 'ESTABLECIMIENTO' in df.columns:
            df['ESTABLECIMIENTO'] = df['ESTABLECIMIENTO'].apply(
                lambda x: reemplazos.get(str(x).upper().strip(), x)
            )

    # Paso 4: Reporte de cambios
    cambios = [{'VARIANTE': k, 'CORREGIDO A': v} for k, v in reemplazos.items()]
    return pd.DataFrame(cambios).drop_duplicates()

correcciones = corregir_nombres_similares(datasets, threshold=0.95)

if not correcciones.empty:
    print("✅ Correcciones aplicadas:")
    display(correcciones)
else:
    print("✅ No se encontraron nombres similares para corregir.")


✅ Correcciones aplicadas:


,VARIANTE,CORREGIDO A
0,COLEGIO CENTRO DE FORMACION PROFESIONAL PREUNI...,COLEGIO CENTRO DE FORMACION PROFESIONAL PREUNI...
1,ESCUELA EN CIENCIAS DE LA COMUNICACION ECCO,ESCUELA EN CIENCIAS DE LA COMUNICACION ECCO II
2,INSTITUTO PARTICULAR DE FORMACION Y DESARROLLO...,INSTITUTO PARTICULAR MIXTO DE FORMACION Y DESA...
3,INTITUTO TECNOLOGICO DE SUR ORIENTE,INSTITUTO TECNOLOGICO DE SURORIENTE
4,INSTITUTO DE EDUCACION BASICA Y BACHILLERATO E...,INSTITUTO DE EDUCACION BASICA Y BACHILLERATO E...
5,CENTRO DE EDUCACION EXTRAESCOLARCEEXGRUPO CEIB...,CENTRO DE EDUCACION EXTRAESCOLARCEEX GRUPO CEI...
6,LICEO DE COMPUTACION SIGLO 21 NO3,LICEO DE COMPUTACION SIGLO 21 NO 3
7,LICEO CLASICO DE COMPUTACION,LICEO CLASICO EN COMPUTACION
8,COLEGIO VALVERDE NO2,COLEGIO VALVERDE NO 2
9,CENTRO TECNOLOGICO DE COMPUTACION LA MERCED,CENTRO TECNOLOGICO DE COMPUTACION LA MERCED S A


Luego de un review manual, llegamos a la conclusión que todos los cambios realizados son verídicos y fueron realizados correctamente. Ahora, podemos revisar que las direcciones sean las mismas para establecimientos que tengan nombres idénticos.

In [679]:
from difflib import get_close_matches
from collections import defaultdict
import pandas as pd

def corregir_direcciones_similares(datasets, threshold=0.95):
    """
    Corrige direcciones similares (pero no idénticas) dentro del mismo establecimiento,
    siempre que la similitud sea mayor al threshold.
    """

    # Paso 1: Agrupar por nombre de establecimiento
    grupos = defaultdict(list)
    for nombre_df, df in datasets.items():
        if all(col in df.columns for col in ['ESTABLECIMIENTO', 'DIRECCION']):
            for idx, row in df.iterrows():
                if pd.notna(row['ESTABLECIMIENTO']) and pd.notna(row['DIRECCION']):
                    clave = str(row['ESTABLECIMIENTO']).upper().strip()
                    direccion = str(row['DIRECCION']).upper().strip()
                    grupos[clave].append((nombre_df, idx, direccion))

    reemplazos = {}

    for establecimiento, lista in grupos.items():
        direcciones_unicas = list(set([direccion for _, _, direccion in lista]))
        if len(direcciones_unicas) <= 1:
            continue  # Nada que comparar

        clusters = []

        for direccion in direcciones_unicas:
            # Solo comparar contra otras que no son iguales
            found_cluster = False
            for cluster in clusters:
                for existente in cluster:
                    if direccion != existente and get_close_matches(direccion, [existente], n=1, cutoff=threshold):
                        cluster.append(direccion)
                        found_cluster = True
                        break
                if found_cluster:
                    break
            if not found_cluster:
                clusters.append([direccion])

        for cluster in clusters:
            if len(cluster) < 2:
                continue
            ref = max(cluster, key=len)  # o más frecuente si prefieres
            for variante in cluster:
                if variante != ref:
                    reemplazos[variante] = ref

    # Paso 3: Reemplazar en datasets
    for df in datasets.values():
        if 'DIRECCION' in df.columns:
            df['DIRECCION'] = df['DIRECCION'].apply(
                lambda x: reemplazos.get(str(x).upper().strip(), x)
            )

    # Paso 4: Reportar cambios
    cambios = [{'VARIANTE': k, 'CORREGIDO A': v} for k, v in reemplazos.items()]
    return pd.DataFrame(cambios).drop_duplicates()

correcciones_direcciones = corregir_direcciones_similares(datasets, threshold=0.95)

if not correcciones_direcciones.empty:
    print("✅ Direcciones corregidas por similitud (excluyendo las ya idénticas):")
    display(correcciones_direcciones)
else:
    print("✅ No se encontraron direcciones similares no idénticas para corregir.")

✅ Direcciones corregidas por similitud (excluyendo las ya idénticas):


,VARIANTE,CORREGIDO A
0,2 AVENIDA 11 CALLE ZONA1,2 AVENIDA 11 CALLE ZONA 1
1,9 CALLEJON B 12-94 ZONA 4 SECTOR LOS OLIVOS EL...,9O CALLEJON B 12-94 ZONA 4 SECTOR LOS OLIVOS E...
2,CAERIO SANTA RITA PACHIPA,CASERIO SANTA RITA PACHIPA
3,CARRETERA INTERAMERICANA P6 ZONA 4,CARRETERA INTERAMERICACANA P6 ZONA 4
4,3 CALLE1-22 ZONA 3,3 CALLE 1-22 ZONA 3
...,...,...
115,1 CALLE CALLEJON LA ALBORADA1-85 ZONA 4,1 CALLE CALLEJON LA ALBORADA 1-85 ZONA 4
116,5CALLE 14-42 ZONA 5,5 CALLE 14-42 ZONA 5
117,1CALLE 2AVENIDA1-04 ZONA 2,1CALLE 2 AVENIDA 1-04 ZONA 2
118,5CALLE 5-92 ZONA 1,5 CALLE 5-92 ZONA 1


Luego de una revisión manual, las direcciones actualizadas contenían errores de digitación y representan una misma dirección. Gracias a estos pasos adicionales, evitamos establecimientos y direcciones con errores de digitación y tenemos una mejor consistencia en nuestros datos.

## Merging de Establecimientos y "Checkpoint"
Los establecimientos tendrán múltiples entradas en el dataset final, esto ya que cada combinación de "Status" "Modalidad" "Jornada" y "Plan" se refiere a su propio y único plan de estudios. Si fuéramos a utilizar un Merge, luego quedaría un dataset donde se asume que el establecimiento ofrece planes de cualquier combinación presente. Por ejemplo, tendríamos un establecimiento con un plan Diario Matutino, y Fin De Semana Vespertino. Luego de encodear las variables, tendríamos datos que nos indican la existencia de un plan Fin de Semana Matutino.

Recapitulando hasta el momento, hemos limpiado las variables hasta Director enfocados en la consistencia de los datos. El dataset actualmente se ve de esta manera:

In [680]:
first_key = next(iter(datasets))
first_df = datasets[first_key]
print(f"📄 First dataset: {first_key}")
display(first_df.head(10))


📄 First dataset: datos_totonicapan


,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,08-01-0106-46,08-001,TOTONICAPAN,TOTONICAPAN,ESCUELA NORMAL NACIONAL RURAL DE OCCIDENTE GUI...,ZONA 5 PARAJE PARRAMON,77661038,MIGUEL AJPOP VASQUEZ,OTTO GILBERTO AMADO TELLO,DIVERSIFICADO,OFICIAL,RURAL,ABIERTA,BILINGUE,MATUTINA,DIARIO(REGULAR),TOTONICAPÁN
1,08-01-0107-46,08-024,TOTONICAPAN,TOTONICAPAN,COLEGIO MIXTO PARTICULAR JUAN FRANKLIN,3 CALLE 12-08 ZONA 2,77661920,MIGUEL AJPOP VASQUEZ,JOSE LUIS IXCAQUIC AGUILAR,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),TOTONICAPÁN
2,08-01-0203-46,08-024,TOTONICAPAN,TOTONICAPAN,COLEGIO PRIVADO MIXTO LA FAMILIA,6 AVENIDA 1-47 ZONA 1,77667184,MIGUEL AJPOP VASQUEZ,JUAN GILBERTO TZIC TZIC,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),TOTONICAPÁN
3,08-01-0212-46,08-024,TOTONICAPAN,TOTONICAPAN,COLEGIO PRIVADO MIXTO EL SHADAI,8A CALLE 5-00 ZONA 4,59238825,MIGUEL AJPOP VASQUEZ,JESUS CRISTINA LOPEZ TZOC,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),TOTONICAPÁN
4,08-01-0214-46,08-024,TOTONICAPAN,TOTONICAPAN,COLEGIO PRIVADO MIXTO LA FAMILIA,6 AVENIDA 1-47 ZONA 1,77667184,MIGUEL AJPOP VASQUEZ,HIGINIO DAVID JUAREZ RODRIGUEZ,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),TOTONICAPÁN
5,08-01-0217-46,08-003,TOTONICAPAN,TOTONICAPAN,INSTITUTO MIXTO DE EDUCACION DIVERSIFICADA POR...,ALDEA CHUATROJ,57377254,OSCAR MOISES LACAN ALVAREZ,VICTOR FELIPE BAQUIAX PUAC,DIVERSIFICADO,COOPERATIVA,RURAL,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),TOTONICAPÁN
6,08-01-0222-46,08-024,TOTONICAPAN,TOTONICAPAN,COLEGIO PRIVADO MIXTO LA FAMILIA,6 AVENIDA 1-47 ZONA 1,77667184,MIGUEL AJPOP VASQUEZ,HIGINIO DAVID JUAREZ RODRIGUEZ,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,DOBLE,DIARIO(REGULAR),TOTONICAPÁN
7,08-01-0232-46,08-024,TOTONICAPAN,TOTONICAPAN,CENTRO DE ESTUDIOS AVANZADOS DE TOTONICAPAN,9 CALLE A 08-043 ZONA 3,56769964-77663283,MIGUEL AJPOP VASQUEZ,LUIS NERY MORALES GARCIA,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,DOBLE,DIARIO(REGULAR),TOTONICAPÁN
8,08-01-0233-46,08-024,TOTONICAPAN,TOTONICAPAN,CENTRO DE ESTUDIOS AVANZADOS DE TOTONICAPAN,9 CALLE A 08-043 ZONA 3,56769964,MIGUEL AJPOP VASQUEZ,LUIS NERY MORALES GARCIA,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),TOTONICAPÁN
9,08-01-0238-46,08-024,TOTONICAPAN,TOTONICAPAN,COLEGIO ADVENTISTA MARANATHA,4 CALLE 0-10 DE LA ZONA 2,77661520,MIGUEL AJPOP VASQUEZ,PATROCINIO ABELINO TZUL BARRENO,DIVERSIFICADO,PRIVADO,RURAL,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),TOTONICAPÁN


De momento, queda pendiente la limpieza de las variables categóricas restantes y eliminación de duplicados. La eliminación de duplicados la estamos manteniendo hasta el final, ya que primero debemos asegurarnos que un caso como "Doble" y "Dóble" no genere datos duplicados debido a errores de ingreso.